In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import os 
from collections import Counter
from tqdm.autonotebook import tqdm 
import warnings

warnings.filterwarnings("ignore")

/home/chen/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from utils import *

In [3]:
import time 

# 1. 基本配置和数据加载

## 1.1 基本配置

In [4]:
class Config(dict):
    def __init__(self):
        self['data_dir'] = "../data/ml-1m/ratings.dat"
        self['data_path'] = "../data/ml_lm"
        self['column_names'] = ['user', 'item']
        
        # 用于训练数据生成的参数
        self['num_neg'] = 10    ## 表示每个user生成的负样本的数量
        self['test_neg'] = 99  ## 测试集每个user的负样本数量
        self['batch_size'] = 256    ## 表示每个batch的大小
        self['topk'] = 10 
        self['seed'] = 2019
        
        # 模型参数
        self['embed_size'] = 20  # 表示嵌入向量
        self['dropout_prob'] = 0.8 
        self['l2_reg'] = 0.001
        
        # 训练参数
        self['num_epochs'] = 2 
        self['evaluateEvery'] = 1000
        self['checkpointEvery'] = 1000
        self['lr'] = 0.01 
        self['decay_steps'] = 200 
        self['decay_rate'] = 0.9 
        self['grad_clip'] = 5.0 
        
        # 其他参数
        self['num_classes'] = 1 
        self['train_size'] = 0.8 
        self.threshold = 0.5 
        self['checkpoint_dir'] = "../model/NCF/checkpoint"
        self['summary_dir'] = "../model/NCF/summary"
        self['max_to_keep'] = 5 

## 1.2 加载数据

In [5]:
class Dataset:
    def __init__(self, config):
        self.config = config
        self.load_data()
    

    def load_data(self):
        full_data = pd.read_csv(self.config['data_dir'], sep="::", header=None, names=self.config['column_names'],
                               usecols=[0,1], dtype={0:np.int32, 1:np.int32}, engine='python')
        full_data["user"] = full_data["user"] - 1 
        user_set = set(full_data['user'].unique())
        item_set = set(full_data['item'].unique())
        # 获取用户的数量和商品的数量
        self.user_size = len(user_set)
        self.item_size = len(item_set)
        ## 对商品进行重新编号
        item_map = self.re_index(item_set)
        item_list = []
        full_data['item'] = full_data['item'].map(lambda x: item_map[x])
        item_set = set(full_data.item.unique())
        ## 记录每个用户有过行为的商品
        self.user_bought = {}
        for i in range(len(full_data)):
            u = full_data['user'][i]
            t = full_data['item'][i]
            if u not in self.user_bought:
                self.user_bought[u] = []
            self.user_bought[u].append(t)
        
        ## 对于每个用户来说的负样本
        self.user_negative = {}
        for key in self.user_bought:
            self.user_negative[key] = list(item_set - set(self.user_bought[key]))
        
        # 划分训练集和测试集
        ## 按照用户进行分组，得到每个用户有过行为的商品的数量
        user_length= full_data.groupby('user').size().tolist()
        split_train_test = []
        
        ## 将每个用户前n-1个作为训练集，第n个作为测试集
        for i in range(len(user_set)):
            for _ in range(user_length[i]-1):
                split_train_test.append("train")
            split_train_test.append("test")
            
        full_data['split'] = split_train_test
        
        train_data = full_data[full_data['split'] == "train"].reset_index(drop=True)
        test_data = full_data[full_data['split']=="test"].reset_index(drop=True)
        del train_data['split']
        del test_data['split']
        
        labels = np.ones(len(train_data), dtype=np.int32)
        ## features是DataFrame结构，labels是ndarray结构
        self.train_features = train_data
        self.train_labels = labels
        
        self.test_features = test_data
        self.test_labels = test_data['item'].values
        
    
    def add_negative(self, features, labels, numbers, is_training):
        feature_user, feature_item, labels_add, feature_dict = [],[],[],{}
        for i in range(len(features)):
            user = features['user'][i]
            item = features['item'][i]
            label = labels[i]
            
            feature_user.append(user)
            feature_item.append(item)
            labels_add.append(label)
            
            ## 添加负样本
            neg_samples = np.random.choice(self.user_negative[user], size=numbers, replace=False)
            
            if is_training:
                for k in neg_samples:
                    feature_user.append(user)
                    feature_item.append(k)
                    labels_add.append(0)
            else:
                for k in neg_samples:
                    feature_user.append(user)
                    feature_item.append(k)
                    labels_add.append(k)
        feature_dict['user'] = feature_user
        feature_dict['item'] = feature_item
        
        return feature_dict, labels_add
    
    def dump_data(self, features, labels,  num_neg, is_training):
        if not os.path.exists(self.config['data_path']):
            os.makedirs(self.config['data_path'])
            
        features, labels = self.add_negative(features,  labels, num_neg, is_training)
        
        data_dict=  dict([('user', features['user']), 
                         ('item', features['item']), 
                         ('label', labels)])
        if is_training:
            np.save(os.path.join(self.config['data_path'], "train_data.npy"), data_dict)
        else:
            np.save(os.path.join(self.config['data_path'], "test_data.npy"), data_dict)
        
    
    
    def re_index(self, x):
        i = 0
        x_map = {}
        for key in x:
            x_map[key] = i
            i += 1 
        return x_map
            

In [6]:
class DataGen:
    def __init__(self, config, dataset):
        self.config = config
        self.dataset = dataset
    
    def train_input_fn(self):
        '''构造训练数据'''
        data_path = os.path.join(self.config['data_path'], 'train_data.npy')
        # 如果不存在则生成数据文件
        if not os.path.exists(data_path):
            self.dataset.dump_data(self.dataset.train_features, self.dataset.train_labels, 
                                   self.config['num_neg'], True)
        ## 加载数据
        data = np.load(data_path, allow_pickle=True).item()
        print("Loading train data finished!")
        datagen = tf.data.Dataset.from_tensor_slices(data)
        datagen = datagen.shuffle(100000).batch(self.config['batch_size'])
        return datagen
    
    def eval_input_fn(self):
        '''构造测试数据'''
        data_path = os.path.join(self.config['data_path'], 'test_data.npy')
        # 如果不存在则生成文件
        if not os.path.exists(data_path):
            self.dataset.dump_data(self.dataset.test_features, self.dataset.test_labels, 
                           self.config['test_neg'], False)
        data = np.load(data_path, allow_pickle=True).item()
        print("Loading test data finished!")
        datagen = tf.data.Dataset.from_tensor_slices(data)
        datagen = datagen.batch(self.config['test_neg']+1)
        return datagen

# 2. 搭建模型

In [7]:
class NCF(BaseModel):
    def __init__(self, config, iterator):
        super().__init__(config)
        tf.set_random_seed(self.config['seed'])
        self.initializer = tf.initializers.glorot_normal()
        self.iterator = iterator
        
        self.get_data()
        self.build_model()
        self.evaluation()
        self.init_saver()

        
    def get_data(self):
        sample = self.iterator.get_next()
        self.user = sample['user']
        self.item = sample['item']
        self.label = tf.cast(sample['label'], tf.float32)
        
    def build_model(self):
        with tf.name_scope("input"):
            self.user_onehot = tf.one_hot(self.user, self.config['user_size'], name="user_onehot")
            self.item_onehot = tf.one_hot(self.item, self.config['item_size'], name='item_onehot')
            self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
            
        with tf.name_scope("embed"):
            self.user_embed_GMF = tf.layers.dense(inputs=self.user_onehot,
                                                 units=self.config['embed_size'],
                                                 activation=tf.nn.relu,
                                                 kernel_initializer=self.initializer,
                                                 name="user_embed_GMF")
            self.item_embed_GMF = tf.layers.dense(inputs=self.item_onehot,
                                                 units=self.config['embed_size'],
                                                 activation=tf.nn.relu,
                                                 kernel_initializer=self.initializer,
                                                 name="item_embed_GMF")
            self.user_embed_MLP = tf.layers.dense(inputs=self.user_onehot,
                                                 units=self.config['embed_size'],
                                                 activation=tf.nn.relu,
                                                 kernel_initializer=self.initializer,
                                                 name="user_embed_MLP")
            self.item_embed_MLP = tf.layers.dense(inputs=self.item_onehot,
                                                 units=self.config['embed_size'],
                                                 kernel_initializer=self.initializer,
                                                 name="item_embed_MLP")
            
        with tf.name_scope("GMF"):
            # 将对应的user和item隐向量相乘
            self.GMF = tf.multiply(self.user_embed_GMF, self.item_embed_GMF, name="GMF")
            
        with tf.name_scope("MLP"):
            # 获取user和item的深层交互特征
            self.interaction = tf.concat([self.user_embed_MLP, self.item_embed_MLP],
                                        axis=-1, name='interaction')
            self.layer1_MLP = tf.layers.dense(inputs=self.interaction,
                                             units=self.config['embed_size']*2,
                                             activation=tf.nn.relu,
                                             kernel_initializer=self.initializer,
                                             name='layer1_MLP')
            self.layer1_MLP = tf.nn.dropout(self.layer1_MLP, self.dropout_keep_prob)
            
            self.layer2_MLP = tf.layers.dense(inputs=self.layer1_MLP,
                                             units=self.config['embed_size'],
                                             activation=tf.nn.relu,
                                             kernel_initializer=self.initializer,
                                             name="layer2_MLP")
            self.layer2_MLP = tf.nn.dropout(self.layer2_MLP, self.dropout_keep_prob)
            
            self.layer3_MLP = tf.layers.dense(inputs=self.layer2_MLP,
                                             units=self.config['embed_size']//2,
                                             activation=tf.nn.relu,
                                             kernel_initializer=self.initializer,
                                             name="layer3_MLP")
            self.layer3_MLP = tf.nn.dropout(self.layer3_MLP, self.dropout_keep_prob)
            
        with tf.name_scope('output'):
            self.concatenation = tf.concat([self.GMF, self.layer3_MLP], axis=-1, name="concatenation")
            self.logits = tf.layers.dense(inputs=self.concatenation,
                                         units=1,
                                         activation=None, 
                                         kernel_initializer=self.initializer,
                                         name="layer4_output")
            self.logits_dense = tf.reshape(self.logits, [-1])
            self.predictions = tf.nn.sigmoid(self.logits_dense)
            
        with tf.name_scope("loss"):
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.label,
                                                                              logits=self.logits_dense))
            #l2_loss = tf.constant(0.0)
            if self.config['l2_reg'] > 0:
                l2_loss = tf.add_n([tf.nn.l2_loss(cand_var) for cand_var in tf.trainable_variables() 
                                   if "layer" in cand_var.name])
                self.loss += self.config['l2_reg'] * l2_loss
                
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            learning_rate = tf.train.exponential_decay(self.config['lr'],
                                                      self.global_step_tensor,
                                                      self.config['decay_steps'],
                                                      self.config['decay_rate'],
                                                      staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(self.loss)
            
            for idx, (grad, var) in enumerate(grads_and_vars):
                if grad is not None:
                    grads_and_vars[idx] = (tf.clip_by_norm(grad, self.config['grad_clip']), var)
            
            self.train_op = optimizer.apply_gradients(grads_and_vars, global_step=self.global_step_tensor)
    
    def evaluation(self):
        with tf.name_scope("evaluation"):
            # 当数据是测试集的时候，所有的item属于同一个用户的
            _, self.indice = tf.nn.top_k(self.predictions, self.config['topk'])
    
    def init_saver(self):
        self.saver = tf.train.Saver(max_to_keep=self.config['max_to_keep'])

# 3. 定义训练类

In [8]:
# 定义metrics
def mrr(gt_item, pred_items):
    if gt_item in pred_items:
        index = np.where(pred_items == gt_item)[0][0]
        ## 计算倒数，表示越靠前值越大
        return np.reciprocal(float(index+1))
    return 0
    
def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1 
    return 0 

def ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = np.where(pred_items == gt_item)[0][0]
        return np.reciprocal(np.log2(index+2))
    return 0

In [9]:
class Trainer(BaseTrain):
    def __init__(self, sess, model, data, config, logger):
        super().__init__(sess, model, data, config, logger)
        self.train = data[0]
        self.eval = data[1]
        
    def train_epoch(self):
        self.sess.run(self.model.iterator.make_initializer(self.train))
        self.model.get_data()
        start_time = time.time()
        print("Train: ")
        try:
            while True:
                loss, step = self.train_step()
                summaries_dict = {"loss": loss}
                self.logger.summarize(step, summarizer='train', scope='train_summary',
                                     summaries_dict=summaries_dict)
                if step % self.config['evaluateEvery'] == 0:
                    print(f"Step: {step}, Loss: {loss}")
        except tf.errors.OutOfRangeError:
            print("Took: {}".format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))

        ## 用测试集对训练得到的模型进行评估
        print("Evaluation: ")
        self.sess.run(self.model.iterator.make_initializer(self.eval))
        self.model.get_data()
        start_time = time.time()
        HR, MRR, NDCG = [], [], []
        try:
            while True:
                prediction, labels = self.eval_step()
                label = int(labels[0])
                ## 计算各项指标
                HR.append(hit(label, prediction))
                MRR.append(mrr(label, prediction))
                NDCG.append(ndcg(label, prediction))
        except tf.errors.OutOfRangeError:
            hr_mean = np.array(HR).mean()
            mrr_mean = np.array(MRR).mean()
            ndcg_mean = np.array(NDCG).mean()
            print("Took: {}".format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))
            print("HR is {:.3f}, MRR is {:.3f}, NDCG is {:.3f}".format(hr_mean, mrr_mean, ndcg_mean))
            summaries_dict = {"HR": hr_mean, "MRR": mrr_mean, "NDCG": ndcg_mean}
            self.logger.summarize(step, summarizer="test", scope="test_summary",
                                 summaries_dict=summaries_dict)
        ## 保存模型
        self.model.save(self.sess)
        

    def train_step(self):
        feed_dict = {self.model.dropout_keep_prob: self.config['dropout_prob']}
        _, loss, step = self.sess.run([self.model.train_op, self.model.loss, self.model.global_step_tensor],
                                  feed_dict=feed_dict)
        return loss, step
        
    def eval_step(self):
        feed_dict = {self.model.dropout_keep_prob: 1.0}
        indice, item = self.sess.run([self.model.indice, self.model.item],
                                    feed_dict = feed_dict)
        ## 按照预测排名列出商品
        #print(np.unique(user))
        prediction = np.take(item, indice)
        return prediction, item

# 4. 训练

In [10]:
def main():
    tf.reset_default_graph()
    config = Config()
    dataset = Dataset(config)
    data = DataGen(config, dataset)
    train_data = data.train_input_fn()
    test_data = data.eval_input_fn()
    
    config['user_size'] = dataset.user_size
    config['item_size'] = dataset.item_size
    create_dirs([config['summary_dir'], config['checkpoint_dir']])

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.8 
    session_conf.gpu_options.allow_growth = True
    sess = tf.Session(config=session_conf)

    iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
    model = NCF(config, iterator)

    print("加载已经存在的模型...")
    model.load(sess)
    
    pack_data = [train_data, test_data]
    logger = Logger(sess, config)

    trainer = Trainer(sess, model, pack_data, config, logger)
    trainer.train_all()

In [11]:
main()

Loading train data finished!
Loading test data finished!
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
加载已经存在的模型...

当前正处于第1次迭代
Train: 
Step: 1000, Loss: 0.37952202558517456
Step: 2000, Loss: 0.38519033789634705
Step: 3000, Loss: 0.3475574254989624
Step: 4000, Loss: 0.36829236149787903
Step: 5000, Loss: 0.39247995615005493
Step: 6000, Loss: 0.4150899648666382
Step: 7000, Loss: 0.30751311779022217
Step: 8000, Loss: 0.41413211822509766
Step: 9000, Loss: 0.3936382532119751
Step: 10000, Loss: 0.33736452460289
Step: 11000, Loss: 0.39479273557662964
Step: 12000, Loss: 0.35984179377555847
Step: 13000, Loss: 0.3317415714263916
Step: 14000, Loss: 0.3633555471897125
Step: 15000, Loss: 0.4040967524051666
Step: 16000, Loss: 0.37653374671936035
Step: 17000, Loss: 0.33586615324020386
Step: 18000, Loss: 